In [1]:
# from ghostnet.ghost_net import ghost_net
# from keras.datasets import mnist
import pandas as pd
import numpy as np
import torch
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn,optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os
from VGG import *
# 检验GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# 加载训练集合(Train)
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                           train=True,
                                           transform=transform,
                                           download=True)
# 加载测试集合(Test)
test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                          train=False,
                                          transform=transform,
                                          download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
batch_size = 10
# 根据数据集定义数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [6]:
from ghostnet.ghost_net import ghost_net
net = ghost_net(width_mult=1.0,num_classes=10).to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
examples = iter(test_loader)
example_data, _ = examples.next()
net(example_data.to(device))[4].shape

torch.Size([10, 10])

In [1]:
num_epochs = 40
n_total_steps = len(train_loader)
LossList = [] # 记录每一个epoch的loss
AccuryList = [] # 每一个epoch的accury
for epoch in range(num_epochs):
    # -------
    # 开始训练
    # -------
    net.train() # 切换为训练模型
    totalLoss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device) # 图片大小转换
        labels = labels.to(device)
        # 正向传播以及损失的求取
        outputs = net(images)
#         labels = labels.squeeze(1)
        optimizer.zero_grad() # 梯度清空
        loss = criterion(outputs, labels)
        loss.backward(retain_graph=True)
        totalLoss = totalLoss + loss.item()
#         print(loss)
        # 反向传播
         # 反向传播
        optimizer.step() # 权重更新
        if (i+1) % 1000 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, n_total_steps, totalLoss/(i+1)))
    LossList.append(totalLoss/(i+1))
    # ---------
    # 开始测试
    # ---------
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1) # 预测的结果
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(correct)
        acc = 100.0 * correct / total # 在测试集上总的准确率
        AccuryList.append(acc)
        print('Accuracy of the network on the {} test images: {} %'.format(total, acc))
print("模型训练完成")